In [21]:
import pandas as pd

In [22]:
df = pd.read_csv('nyc-rolling-sales.csv')

#drop unnecessary columns 
new_df = df.drop(['Unnamed: 0', 'EASE-MENT', 'APARTMENT NUMBER', 'ADDRESS'], axis=1)

In [23]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   BOROUGH                         84548 non-null  int64 
 1   NEIGHBORHOOD                    84548 non-null  object
 2   BUILDING CLASS CATEGORY         84548 non-null  object
 3   TAX CLASS AT PRESENT            84548 non-null  object
 4   BLOCK                           84548 non-null  int64 
 5   LOT                             84548 non-null  int64 
 6   BUILDING CLASS AT PRESENT       84548 non-null  object
 7   ZIP CODE                        84548 non-null  int64 
 8   RESIDENTIAL UNITS               84548 non-null  int64 
 9   COMMERCIAL UNITS                84548 non-null  int64 
 10  TOTAL UNITS                     84548 non-null  int64 
 11  LAND SQUARE FEET                84548 non-null  object
 12  GROSS SQUARE FEET               84548 non-null

In [24]:
new_df.describe(include = 'all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
BOROUGH,84548.0,NaN,NaN,NaN,2.998758,1.28979,1.0,2.0,3.0,4.0,5.0
NEIGHBORHOOD,84548,254,FLUSHING-NORTH,3068,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BUILDING CLASS CATEGORY,84548,47,01 ONE FAMILY DWELLINGS,18235,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TAX CLASS AT PRESENT,84548,11,1,38633,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BLOCK,84548.0,NaN,NaN,NaN,4237.218976,3568.263407,1.0,1322.75,3311.0,6281.0,16322.0
LOT,84548.0,NaN,NaN,NaN,376.224015,658.136814,1.0,22.0,50.0,1001.0,9106.0
BUILDING CLASS AT PRESENT,84548,167,D4,12663,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP CODE,84548.0,NaN,NaN,NaN,10731.991614,1290.879147,0.0,10305.0,11209.0,11357.0,11694.0
RESIDENTIAL UNITS,84548.0,NaN,NaN,NaN,2.025264,16.721037,0.0,0.0,1.0,2.0,1844.0
COMMERCIAL UNITS,84548.0,NaN,NaN,NaN,0.193559,8.713183,0.0,0.0,0.0,0.0,2261.0


In [25]:
#change numberic values to numeric data type
new_df['SALE PRICE'] = pd.to_numeric(new_df['SALE PRICE'], errors = 'coerce')
new_df['LAND SQUARE FEET'] = pd.to_numeric(new_df['LAND SQUARE FEET'], errors = 'coerce')
new_df['GROSS SQUARE FEET'] = pd.to_numeric(new_df['GROSS SQUARE FEET'], errors = 'coerce')


In [27]:
#drop intances where sale price is null
new_df = new_df[new_df['SALE PRICE'].notna()]

In [28]:
new_df.shape

(69987, 18)

In [29]:
new_df['SALE DATE'].head()

0    2017-07-19 00:00:00
3    2016-09-23 00:00:00
4    2016-11-17 00:00:00
6    2016-09-23 00:00:00
9    2016-11-07 00:00:00
Name: SALE DATE, dtype: object

In [30]:
#remove timestamp from sale date with lambda function
new_df['SALE DATE'] = new_df['SALE DATE'].apply(lambda x: x.split(' ')[0])
new_df['SALE DATE'].head()

0    2017-07-19
3    2016-09-23
4    2016-11-17
6    2016-09-23
9    2016-11-07
Name: SALE DATE, dtype: object

In [31]:
#sale price still has nonsencical values on low end 
new_df = new_df[new_df['SALE PRICE'] > 10000]

In [32]:
#find outlier bounds for sale price
q_low = new_df['SALE PRICE'].quantile(0.01)
q_high = new_df['SALE PRICE'].quantile(0.99)

print(q_low)
print(q_high)

50000.0
14100000.0


In [33]:
#remove sales price outliers
new_df = new_df[(new_df['SALE PRICE'] > q_low) & (new_df['SALE PRICE'] < q_high)]

In [34]:
print(new_df['SALE PRICE'].min())
print(new_df['SALE PRICE'].max())

50200.0
14000000.0


In [35]:
#create age column using year built
new_df['age'] = new_df['YEAR BUILT'].apply(lambda x: x if x < 1 else 2023 - x)
new_df.age.head()

0     123
3     110
4     123
6     103
11     14
Name: age, dtype: int64

In [36]:
#export
new_df.to_csv('building_data_cleaned', index = False)